### 네이버뉴스 크롤링 최종
 - 섹션 추출 ~ sub페이지 추출까지


In [1]:
from urllib.request import urlopen
import bs4
import requests
import pandas as pd

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

### (1) 섹션 메뉴및 링크 추출

In [3]:
url = 'https://news.naver.com/'
html = urlopen(url)
bs_obj = bs4.BeautifulSoup(html, 'html.parser')

In [4]:
ul = bs_obj.select_one('ul.Nlnb_menu_list')

section = []
link = []
for lis in ul.select('li'):
    section.append(lis.find('span').text)
    link.append(lis.find('a')['href'])

In [5]:
section_menu_df = pd.DataFrame({'section':section, 'link':link})
section_menu_df.to_csv('crawl_data/naver_news_section.csv', index=0)

### (2) 토픽 뉴스및 링크 추출

In [6]:
section_menu_df = pd.read_csv('crawl_data/naver_news_section.csv')

In [7]:
def get_topic(url, section):
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')
    
    topic_list = bs_obj.findAll('div', {'class':'sh_text'})   
    topic_title = []
    topic_link = []

    for topic in topic_list:
        try:
            a = topic.find('a')
            topic_title.append(a.text)
            topic_link.append(a['href'])
        except:
            print('추출시 오류 발생')
            
    return {'topic':topic_title, 'url':topic_link, 'section':section}

In [8]:
all_topic_df = pd.DataFrame({
    'topic':[],
    'url':[],
    'section':[]
})

for i in range(1, 7):
    df = pd.DataFrame(get_topic(section_menu_df['link'][i], section_menu_df['section'][i]))
    all_topic_df = pd.concat([all_topic_df, df], axis=0, ignore_index=True)
    
all_topic_df.to_csv('crawl_data/naver_news_all_topic.csv')

### (3) 토픽 뉴스 내용 추출

In [9]:
all_topic_df = pd.read_csv('crawl_data/naver_news_all_topic.csv')

In [10]:
def get_sub_news_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    paper_list, title_list, datetime_list, writing_list, writer_list = [[] for _ in range(5)]
    try:
        res = requests.get(url, headers=headers)
        html = res.text
        bs_obj = bs4.BeautifulSoup(html, 'html.parser')
        
        paper_list.append(bs_obj.find('a', {'class':'media_end_head_top_logo'}).select_one('img')['title'])
        title_list.append(bs_obj.find('div', {'class':'media_end_head_title'}).find('span').text)
        datetime_list.append(bs_obj.find('div', {'class':'media_end_head_info_datestamp_bunch'}).find('span').text)
        writing_list.append(bs_obj.find('article', {'id':'dic_area'}).text.replace('\n', ' ').strip())
        writer_list.append(bs_obj.find('div', {'class':'byline'}).text.strip())
    except Exception as e:
        print('오류 발생!', e)
    
    sub_news_dict = {
        'paper' : paper_list,
        'title' : title_list,
        'datetime' : datetime_list,
        'writing' : writing_list,
        'writer' : writer_list
    }
    return sub_news_dict

In [11]:
all_sub_news_info_df = pd.DataFrame({
    'paper' : [],
    'title' : [],
    'datetime' : [],
    'writing' : [],
    'writer' : []
})

for url in all_topic_df['url']:
    df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df = pd.concat([all_sub_news_info_df, df], axis=0, ignore_index=True)
    
all_sub_news_info_df.to_csv('crawl_data/naver_sub_news_section.csv', index=0)